<a href="https://colab.research.google.com/github/oilportrait/test_colab/blob/main/knn_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

targetUrl = "https://raw.githubusercontent.com/oilportrait/test_colab/main/fake_bills.csv"

df2 = pd.read_csv(targetUrl, sep=';')


genuine = df2["is_genuine"]
 
encoded_genuine = pd.get_dummies(genuine, prefix='genuine')
df2 = pd.concat([df2, encoded_genuine], axis = 1)
df2 = df2.drop(df2.columns[0], axis=1)
df2.head()

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt

msno.matrix(df2)
plt.show()

In [ ]:
msno.bar(df2)
plt.show()

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)

df2[["margin_low"]] = imputer.fit_transform(df2[["margin_low"]])
msno.matrix(df2)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( df2.iloc[:,0:6], df2.iloc[:,6:8], stratify=df2.iloc[:,6:8], random_state=1)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
type(y_test)

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt

df_cat = pd.concat([X_train,y_train], axis=1)

df_cat.head()

In [ ]:
sns.pairplot(df_cat, hue='genuine_True' );
plt.show()

In [ ]:
import numpy as np

corr_mat=np.corrcoef(X_train,rowvar=False)

sns.heatmap(corr_mat,linewidth=1,cmap='binary')
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std.fit(X_train.iloc[:,0:])
X_train.iloc[:,0:]= std.transform(X_train.iloc[:,0:])
X_test.iloc[:,0:]= std.transform(X_test.iloc[:,0:])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold

grid_params = {
    'n_neighbors' : list(range(1,20)),
    'weights' : ["uniform", "distance"],
    'metric' : ['euclidean', 'manhattan', 'minkowski']
}
kfold = KFold(n_splits=10, shuffle=True, random_state=0)

gs = GridSearchCV(knn, grid_params, cv=kfold)
gs.fit(X_train, y_train)

print("Best Parameters : ", gs.best_params_)
print("Best Score : ", gs.best_score_)
print("Best Test Score : ", gs.score(X_test, y_test))

In [ ]:
knn_best = KNeighborsClassifier(metric = 'euclidean', n_neighbors = 10, weights = 'uniform')
knn_best.fit(X_train, y_train)

In [ ]:
print('*** Test Accuracy *** \n   {}'.format(knn_best.score(X_test, y_test)))